# Model 1: Tipus incidència 'No ha impartit classe a aquest grup'
## Part-of-speech method

En este notebook se va ha intendar mejorar el modelo de detección de la incidencia 
'No ha impartit classe a aquest grup' mediante el uso de Part-of-Speech y Dependency Parsing.

Aquí se definirán las reglas para classificar las classes, observando los ejemplos.


In [1]:
import json
import os.path
import sys
import csv
import numpy as np
import pandas as pd
from colorama import Fore

In [2]:
from __future__ import unicode_literals, print_function
import plac
import random
from pathlib import Path

In [3]:
import spacy
from spacy.util import minibatch, compounding
from spacy import displacy
from spacy.matcher import Matcher

In [4]:
def load_data (language):
    
    if (debug >= 1):
        print ("LOAD_DATA")
        
    # Load data from file
    file = "comentaris_" + language + ".csv"
    data = pd.read_csv(pathdest + file)
    if (debug >= 1):
        print ("Original data:", data.shape[0])
    if (debug >= 2):
        display (data.sample(5))

    data_profe = data[data.TipusPregunta == "P"]
    data_true = data_profe[data_profe.TipusIncidencia == "No ha impartit classe a aquest grup"][["Comentari"]]
    if (debug >= 1):
        print ("True cases:", data_true.shape[0])
    if (debug >= 2):
        display (data_true.sample(5))    
    
    data_counts = data_true['Comentari'].value_counts().index
    if (debug >= 1):
        print (data_counts[:5])
    
#    display (data_counts.index)
    
#    data_group = data_true.groupby(['Comentari']).count()
#    display (data_group)
    
    #display (data_group.sort_values('counts'))
    
    return data_counts

In [5]:
def show_parts_of_speech(doc):
    
    print ("Text: ", doc)
    print ()
    print ('Parts of speech:')
    print ("TOKEN, TAG, POS, DEP, LEMMA, SHAPE, EXPLAIN")
    for token in doc:
        print (token, token.tag_, token.pos_, token.dep_, token.lemma_, token.shape_, spacy.explain(token.tag_))
    
    print ()
    print ('Dependency parsing:')
    for chunk in doc.noun_chunks:
        print (chunk.text, chunk.root.text, chunk.root.dep_, chunk.root.head.text)
        
    displacy.render(doc, style='dep', jupyter=True, options={'distance':90})


In [6]:
pathori = "../data/original"
pathdest = "../data/preprocessed/"
pathmodel = "../data/models/"
debug = 1

In [7]:
languages = {"ca":"ca_fasttext_wiki", "es":"es_core_news_lg", "en":"en_core_web_sm" }

language = "es"
model = languages["es"]

data_true = load_data (language)


LOAD_DATA
Original data: 1761
True cases: 38
Index(['No he tratado con él', 'No lo he tenido',
       'no hemos recibido ni una clase por su parte en toda la cuarentena, ni suyas ni del resto de profesores. ',
       'No nos dio clases.', 'No se recibió clases con ella'],
      dtype='object')


In [8]:
if (debug == 1):
    print ("PROCESS DATA: ")
    print ("Language: ", language, ", Model: ", model)

# Load the model form spacy
nlp = spacy.load(model)

PROCESS DATA: 
Language:  es , Model:  es_core_news_lg


In [39]:
doc = nlp(data_true[0])
show_parts_of_speech(doc)

doc2 = nlp("No fué nunca a clases")
#doc2 = nlp(data_true[1])
show_parts_of_speech(doc2)

score = doc.similarity(doc2)
print ("Similarity: ", score)

Text:  No he tratado con él

Parts of speech:
TOKEN, TAG, POS, DEP, LEMMA, SHAPE, EXPLAIN
No ADV__Polarity=Neg ADV advmod No Xx None
he AUX__Mood=Ind|Number=Sing|Person=1|Tense=Pres|VerbForm=Fin AUX aux haber xx None
tratado VERB__Gender=Masc|Number=Sing|Tense=Past|VerbForm=Part VERB ROOT tratar xxxx None
con ADP__AdpType=Prep ADP case con xxx None
él PRON__Case=Acc,Nom|Gender=Masc|Number=Sing|Person=3|PronType=Prs PRON obl él xx None

Dependency parsing:
él él obl tratado


Text:  No fué nunca a clases

Parts of speech:
TOKEN, TAG, POS, DEP, LEMMA, SHAPE, EXPLAIN
No ADV__Polarity=Neg ADV advmod No Xx None
fué AUX__Mood=Ind|Number=Sing|Person=3|Tense=Past|VerbForm=Fin AUX ROOT fué xxx None
nunca ADV ADV advmod nunca xxxx adverb
a ADP__AdpType=Prep ADP case a x None
clases NOUN__Gender=Fem|Number=Plur NOUN obl clase xxxx None

Dependency parsing:
clases clases obl fué


Similarity:  0.5315306632076054


In [36]:
print ()
print ("Matches:")

# Rule-based matching
matcher = Matcher(nlp.vocab, validate=True)


pattern = [{"TAG": "ADV__Polarity=Neg"}, {"OP": "*"}, 
           {"POS": "VERB", "LEMMA": {"IN": ["tratar","recibir","tener","dar","dictar","ir","conocer",
                                            "realizar","ver","saber","ser"]}}, 
           {"OP": "*"}, {"POS": "PRON","OP": "?"}]
# Example: "No he tratado", "No traté", "No he recibido", "No recibí"
#pattern = [{"POS":"VERB"}]
matcher.add("NoClasses1", None, pattern)



#doc = nlp("Hello, world! Hello world!")

print ("Text: ", doc)
matches = matcher(doc)
for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]  # Get string representation
    span = doc[start:end]  # The matched span
    print(match_id, string_id, start, end, span.text)
    
#doc2 = nlp("No le he tratado con él")

for text in data_true:
    doc = nlp(text)

    print ()
    print ("Text: ", doc)
    matches = matcher(doc)

    if (len(matches) == 0):
        print ("No matches")
    else:
        match_id, start, end = matches[len(matches)-1]
        string_id = nlp.vocab.strings[match_id]  # Get string representation
        span = doc[start:end]  # The matched span
        print(match_id, string_id, start, end, span.text)
        
        #for match_id, start, end in matches:
        #    string_id = nlp.vocab.strings[match_id]  # Get string representation
        #    span = doc[start:end]  # The matched span
        #    print(match_id, string_id, start, end, span.text)
    



Matches:
Text:  No he tratado con él
6239189525466108865 NoClasses1 0 3 No he tratado
6239189525466108865 NoClasses1 0 4 No he tratado con
6239189525466108865 NoClasses1 0 5 No he tratado con él

Text:  No he tratado con él
6239189525466108865 NoClasses1 0 5 No he tratado con él

Text:  No lo he tenido
6239189525466108865 NoClasses1 0 4 No lo he tenido

Text:  no hemos recibido ni una clase por su parte en toda la cuarentena, ni suyas ni del resto de profesores. 
6239189525466108865 NoClasses1 0 22 no hemos recibido ni una clase por su parte en toda la cuarentena, ni suyas ni del resto de profesores.

Text:  No nos dio clases.
6239189525466108865 NoClasses1 0 5 No nos dio clases.

Text:  No se recibió clases con ella
6239189525466108865 NoClasses1 0 6 No se recibió clases con ella

Text:  La clase no la dictó ella. 
6239189525466108865 NoClasses1 2 7 no la dictó ella.

Text:  No fue nunca a clases
No matches

Text:  No puedo valorar a a este profesor puesto que no ha aparecido en prac